In [1]:
MISMATCH_COST = 2
GAP_COST = 1

In [2]:
"""
Edit distance using recursive approach
"""

import functools

@functools.lru_cache(maxsize=None)
def get_cost(a, b):
    if len(a) == 0:
        return len(b) * GAP_COST
    if len(b) == 0:
        return len(a) * GAP_COST
    
    if a[0] == b[0]:
        return get_cost(a[1:], b[1:])
    else:
        return min(
            get_cost(a[1:], b[1:]) + MISMATCH_COST,
            get_cost(a, b[1:]) + GAP_COST,
            get_cost(a[1:], b) + GAP_COST
        )
    
def print_edit_join(a, b):
    if not a and not b:
        pass
    elif not a:
        print("{}  {}".format("-", b[0]))
        print_edit_join(a, b[1:])
    elif not b:
        print("{}  {}".format(a[0], "-"))
        print_edit_join(a[1:], b)
    elif a[0] == b[0]:
        print("{}  {}".format(a[0], b[0]))
        print_edit_join(a[1:], b[1:])
    else:
        mismatch_cost = get_cost(a[1:], b[1:]) + MISMATCH_COST
        a_gap_cost = get_cost(a, b[1:]) + GAP_COST
        b_gap_cost = get_cost(a[1:], b) + GAP_COST
        
        if mismatch_cost < a_gap_cost and mismatch_cost < b_gap_cost:
            print("{}  {}".format(a[0], b[0]))
            print_edit_join(a[1:], b[1:])
        elif a_gap_cost < b_gap_cost:
            print("{}  {}".format("-", b[0]))
            print_edit_join(a, b[1:])
        else:
            print("{}  {}".format(a[0], "-"))
            print_edit_join(a[1:], b)

print_edit_join("corect", "correct")

c  c
o  o
r  r
-  r
e  e
c  c
t  t


In [9]:
"""
Classig Edit distance algorithm
"""

def dp_edit_distance(str_a, str_b):
    subsol = [[None for _ in range(len(str_b) + 1)] for _ in range(len(str_a) + 1)]
    
    """
    Populate the first row and col (represents the case where there
    is absolutely no commonality between the two strings)
    """
    for i in range(len(str_b) + 1):
        subsol[0][i] = i * GAP_COST
    for i in range(len(str_a) + 1):
        subsol[i][0] = i * GAP_COST
    
    for a_idx in range(len(str_a)):
        for b_idx in range(len(str_b)):
            if str_a[a_idx] == str_b[b_idx]:
                match_cost = subsol[a_idx][b_idx]
            else:
                match_cost = subsol[a_idx][b_idx] + MISMATCH_COST
                
            a_gap_cost = subsol[a_idx][b_idx + 1] + GAP_COST
            b_gap_cost = subsol[a_idx + 1][b_idx] + GAP_COST
            
            if match_cost < a_gap_cost and match_cost < b_gap_cost:
                subsol[a_idx + 1][b_idx + 1] = match_cost
            elif a_gap_cost < b_gap_cost:
                subsol[a_idx + 1][b_idx + 1] = a_gap_cost
            else:
                subsol[a_idx + 1][b_idx + 1] = b_gap_cost
                
    a_idx = len(str_a)
    b_idx = len(str_b)
    fmtstring = "{a_char}  {b_char}"
    print(fmtstring.format(a_char="A", b_char="B"))
    print("-" * len(fmtstring.format(a_char="A", b_char="B")))
    while a_idx > 0 or b_idx > 0:
        if a_idx == 0:
            b_idx -= 1
            print(fmtstring.format(a_char="-", b_char=str_b[b_idx]))
        elif b_idx == 0:
            a_idx -= 1
            print(fmtstring.format(a_char=str_a[a_idx], b_char="-"))
        else:
            match_cost = subsol[a_idx - 1][b_idx - 1] + (
                MISMATCH_COST if str_a[a_idx - 1] != str_b[b_idx - 1] else 0)
            a_gap_cost = subsol[a_idx - 1][b_idx] + GAP_COST
            b_gap_cost = subsol[a_idx][b_idx - 1] + GAP_COST
            
            if match_cost < a_gap_cost and match_cost < b_gap_cost:
                a_idx -= 1
                b_idx -= 1
                print(fmtstring.format(a_char=str_a[a_idx], b_char=str_b[b_idx]))
            elif a_gap_cost < b_gap_cost:
                a_idx -= 1
                print(fmtstring.format(a_char=str_a[a_idx], b_char="-"))
            else:
                b_idx -= 1
                print(fmtstring.format(a_char="-", b_char=str_b[b_idx]))
                
dp_edit_distance("apple", "aple")

A  B
----
e  e
l  l
p  -
p  p
a  a
